In [1]:
using Pkg
Pkg.activate(".")

#relevant toml files have already been added to this environment

  Activating environment at `~/physics-215-julia/session-1/Project.toml`


## KR1: Using @code_* 

For this section, we will explore the @code_* macros and what they can tell about how Julia handles the codes we write.

### @code_native
Following the [Julia documentation](https://docs.julialang.org/en/v1/stdlib/InteractiveUtils/#InteractiveUtils.code_native), the function for the macro `@code_native` prints out the assembly instructions needed by Julia to run the function that is passed through it. We illustrate how it handles this with the following examples. The first cell runs the macro for the `cos(x)` function with input `0`, while the second cell does the same but for input `0.0`.

In [2]:
@code_native cos(0)

	.text
; ┌ @ math.jl:404 within `cos'
	pushq	%rax
; │ @ math.jl:405 within `cos'
; │┌ @ float.jl:206 within `float'
; ││┌ @ float.jl:191 within `AbstractFloat'
; │││┌ @ float.jl:94 within `Float64'
	vcvtsi2sd	%rdi, %xmm0, %xmm0
; │└└└
; │ @ math.jl:407 within `cos'
	movabsq	$cos, %rax
	callq	*%rax
	popq	%rax
	retq
	nopw	%cs:(%rax,%rax)
; └


In [3]:
@code_native cos(0.0)

	.text
; ┌ @ trig.jl:98 within `cos'
	subq	$24, %rsp
	movabsq	$.rodata.cst16, %rax
; │ @ trig.jl:99 within `cos'
; │┌ @ float.jl:441 within `abs'
	vandpd	(%rax), %xmm0, %xmm1
	movabsq	$.rodata.cst8, %rax
	vmovsd	(%rax), %xmm2                   # xmm2 = mem[0],zero
; │└
; │ @ trig.jl:100 within `cos'
; │┌ @ float.jl:371 within `<'
	vucomisd	%xmm1, %xmm2
; │└
	ja	L157
; │ @ trig.jl:105 within `cos'
; │┌ @ float.jl:449 within `isnan'
; ││┌ @ float.jl:368 within `!='
	vucomisd	%xmm0, %xmm0
; │└└
	jp	L2472
; │ @ trig.jl:107 within `cos'
; │┌ @ float.jl:461 within `isinf'
; ││┌ @ float.jl:452 within `isfinite'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm0, %xmm0, %xmm1
	vxorpd	%xmm2, %xmm2, %xmm2
; │││└
; │││┌ @ float.jl:401 within `==' @ float.jl:365
	vucomisd	%xmm2, %xmm1
; │└└└
	jne	L2491
	jp	L2491
; │ @ trig.jl:110 within `cos'
; │┌ @ rem_pio2.jl:221 within `rem_pio2_kernel'
; ││┌ @ math.jl:1188 within `poshighword'
; │││┌ @ essentials.jl:422 within `reinterpret'
	vmovq	%xmm0, %rcx
; ││

	movabsq	$140204254408520, %rcx          # imm = 0x7F83D8C7AF48
; ││└└
; ││┌ @ rem_pio2.jl:70 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:339 within `muladd'
	vfmadd231sd	(%rcx), %xmm2, %xmm0    # xmm0 = (xmm2 * mem) + xmm0
	movabsq	$140204254408544, %rcx          # imm = 0x7F83D8C7AF60
; ││└└
; ││┌ @ rem_pio2.jl:71 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:332 within `*'
	vmulsd	(%rcx), %xmm2, %xmm3
; ││└└
; ││┌ @ rem_pio2.jl:72 within `cody_waite_ext_pio2'
; │││┌ @ int.jl:462 within `>>' @ int.jl:456
	shrl	$20, %eax
; ││└└
; ││┌ @ rem_pio2.jl:73 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm3, %xmm0, %xmm1
; ││└└
; ││┌ @ rem_pio2.jl:74 within `cody_waite_ext_pio2'
; │││┌ @ math.jl:1175 within `highword'
; ││││┌ @ essentials.jl:422 within `reinterpret'
	vmovq	%xmm1, %rcx
; ││└└└
; ││┌ @ rem_pio2.jl:75 within `cody_waite_ext_pio2'
; │││┌ @ int.jl:462 within `>>' @ int.jl:456
	shrq	$52, %rcx
; │││└
; │││┌ @ int.jl:923 within `&' @ int.jl:309
	andl	$20

; ││││┌ @ essentials.jl:422 within `reinterpret'
	vmovq	%xmm1, %rcx
; ││└└└
; ││┌ @ rem_pio2.jl:83 within `cody_waite_ext_pio2'
; │││┌ @ int.jl:462 within `>>' @ int.jl:456
	shrq	$52, %rcx
; │││└
; │││┌ @ int.jl:923 within `&' @ int.jl:309
	andl	$2047, %ecx                     # imm = 0x7FF
; │││└
; │││┌ @ int.jl:86 within `-'
	subl	%ecx, %eax
; ││└└
; ││┌ @ rem_pio2.jl:84 within `cody_waite_ext_pio2'
; │││┌ @ operators.jl:305 within `>'
; ││││┌ @ int.jl:447 within `<' @ promotion.jl:361 @ int.jl:441
	cmpl	$50, %eax
; │││└└
	jae	L1641
; │└└
; │┌ @ rem_pio2.jl within `rem_pio2_kernel'
	vmovapd	%xmm4, %xmm0
	jmp	L1686
L732:
	vxorpd	%xmm1, %xmm1, %xmm1
; │└
; │┌ @ rem_pio2.jl:231 within `rem_pio2_kernel'
; ││┌ @ int.jl:443 within `<='
	cmpl	$1073928572, %eax               # imm = 0x4002D97C
; ││└
	jbe	L1146
; │└
; │┌ @ rem_pio2.jl within `rem_pio2_kernel'
	vucomisd	%xmm1, %xmm0
; │└
; │┌ @ rem_pio2.jl:239 within `rem_pio2_kernel'
	ja	L1323
; │└
; │┌ @ rem_pio2.jl within `rem_pio2_kernel'


	movabsq	$140204254408544, %rax          # imm = 0x7F83D8C7AF60
; ││└└
; ││┌ @ rem_pio2.jl:71 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:332 within `*'
	vmulsd	(%rax), %xmm2, %xmm3
; ││└└
; ││┌ @ rem_pio2.jl:73 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm3, %xmm0, %xmm1
; ││└└
; ││┌ @ rem_pio2.jl:74 within `cody_waite_ext_pio2'
; │││┌ @ math.jl:1175 within `highword'
; ││││┌ @ essentials.jl:422 within `reinterpret'
	vmovq	%xmm1, %rcx
; ││└└└
; ││┌ @ rem_pio2.jl:75 within `cody_waite_ext_pio2'
; │││┌ @ int.jl:462 within `>>' @ int.jl:456
	shrq	$52, %rcx
; │││└
; │││┌ @ int.jl:923 within `&' @ int.jl:309
	andl	$2047, %ecx                     # imm = 0x7FF
	movl	$1025, %eax                     # imm = 0x401
; │││└
; │││┌ @ int.jl:86 within `-'
	movl	$1025, %edx                     # imm = 0x401
	subl	%ecx, %edx
; ││└└
; ││┌ @ rem_pio2.jl:76 within `cody_waite_ext_pio2'
; │││┌ @ operators.jl:305 within `>'
; ││││┌ @ int.jl:447 within `<' @ promotion.jl:361

	movabsq	$140204254408552, %rax          # imm = 0x7F83D8C7AF68
; │└
; │┌ @ rem_pio2.jl:240 within `rem_pio2_kernel'
; ││┌ @ rem_pio2.jl:51 within `cody_waite_2c_pio2'
; │││┌ @ float.jl:339 within `muladd'
	vaddsd	(%rax), %xmm0, %xmm0
	movabsq	$140204254408560, %rax          # imm = 0x7F83D8C7AF70
	vmovsd	(%rax), %xmm2                   # xmm2 = mem[0],zero
; ││└└
; ││┌ @ rem_pio2.jl:52 within `cody_waite_2c_pio2'
; │││┌ @ float.jl:339 within `muladd'
	vaddsd	%xmm2, %xmm0, %xmm1
; ││└└
; ││┌ @ rem_pio2.jl:53 within `cody_waite_2c_pio2'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm1, %xmm0, %xmm0
; │││└
; │││┌ @ float.jl:339 within `muladd'
	vaddsd	%xmm2, %xmm0, %xmm0
; │└└└
; │ @ trig.jl within `cos'
L1363:
	vmulsd	%xmm1, %xmm1, %xmm2
	vmulsd	%xmm2, %xmm2, %xmm3
	jmp	L2143
L1376:
	vxorpd	%xmm1, %xmm1, %xmm1
; │ @ trig.jl:110 within `cos'
; │┌ @ rem_pio2.jl:255 within `rem_pio2_kernel'
; ││┌ @ operators.jl:305 within `>'
; │││┌ @ float.jl:371 within `<'
	vucomisd	%xmm1, %xmm0
; ││└└
	ja

; ││┌ @ rem_pio2.jl:88 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm0, %xmm4, %xmm3
	vsubsd	%xmm1, %xmm3, %xmm3
	movabsq	$140204254408616, %rax          # imm = 0x7F83D8C7AFA8
; │││└
; │││┌ @ float.jl:339 within `muladd'
	vfmsub231sd	(%rax), %xmm2, %xmm3    # xmm3 = (xmm2 * mem) - xmm3
; ││└└
; ││┌ @ rem_pio2.jl:89 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:329 within `-'
	vsubsd	%xmm3, %xmm0, %xmm1
; ││└└
; ││┌ @ rem_pio2.jl:92 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:329 within `-'
L1686:
	vsubsd	%xmm1, %xmm0, %xmm0
	vsubsd	%xmm3, %xmm0, %xmm0
; ││└└
; ││┌ @ rem_pio2.jl:93 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:239 within `unsafe_trunc'
	vcvttsd2si	%xmm2, %rax
; ││└└
	jmp	L1759
; │└
; │┌ @ rem_pio2.jl within `rem_pio2_kernel'
L1701:
	movabsq	$140204254408608, %rax          # imm = 0x7F83D8C7AFA0
; │└
; │┌ @ rem_pio2.jl:252 within `rem_pio2_kernel'
; ││┌ @ rem_pio2.jl:86 within `cody_waite_ext_pio2'
; │││┌ @ float.jl:332 within `*'
	vmuls

	vmulsd	%xmm5, %xmm2, %xmm5
	vmulsd	%xmm4, %xmm5, %xmm4
; ││└
; ││┌ @ float.jl:326 within `+'
	vaddsd	%xmm3, %xmm4, %xmm3
; │└└
; │┌ @ trig.jl:73 within `sin_kernel'
; ││┌ @ float.jl:332 within `*'
	vmulsd	%xmm2, %xmm1, %xmm4
	movabsq	$140204254408504, %rax          # imm = 0x7F83D8C7AF38
; │└└
; │┌ @ trig.jl:74 within `sin_kernel'
; ││┌ @ float.jl:332 within `*'
	vmulsd	(%rax), %xmm0, %xmm5
	vmulsd	%xmm3, %xmm4, %xmm3
; ││└
; ││┌ @ float.jl:329 within `-'
	vsubsd	%xmm3, %xmm5, %xmm3
; ││└
; ││┌ @ float.jl:332 within `*'
	vmulsd	%xmm3, %xmm2, %xmm2
; ││└
; ││┌ @ float.jl:329 within `-'
	vsubsd	%xmm0, %xmm2, %xmm0
	movabsq	$140204254408728, %rax          # imm = 0x7F83D8C7B018
; ││└
; ││┌ @ float.jl:332 within `*'
	vmulsd	(%rax), %xmm4, %xmm2
; ││└
; ││┌ @ float.jl:329 within `-'
	vaddsd	%xmm2, %xmm0, %xmm0
	vsubsd	%xmm0, %xmm1, %xmm0
	movabsq	$140204254408416, %rax          # imm = 0x7F83D8C7AEE0
; │└└
; │┌ @ float.jl:320 within `-'
	vxorpd	(%rax), %xmm0, %xmm0
; │└
	addq	$24, %rsp
	re

; ││┌ @ float.jl:329 within `-'
	vaddsd	%xmm2, %xmm0, %xmm0
	vsubsd	%xmm0, %xmm1, %xmm0
; │└└
	addq	$24, %rsp
	retq
; │ @ trig.jl within `cos'
L2472:
	movabsq	$140204254408512, %rax          # imm = 0x7F83D8C7AF40
	vmovsd	(%rax), %xmm0                   # xmm0 = mem[0],zero
; │ @ trig.jl:102 within `cos'
	addq	$24, %rsp
	retq
; │ @ trig.jl:108 within `cos'
L2491:
	movabsq	$cos_domain_error, %rax
	callq	*%rax
	ud2
	nopl	(%rax)
; └


From here we can immediately see two distinct differences in the outputs. The most obvious one is the length, with `cos(0.0)` needing nearly ten times as many instructions to handle the operation as opposed to just `cos(0)`. The second difference is in the modules used to call the cosine function. `cos(0)` calls its cosine function from the module `math.jl`, while `cos(0.0)` calls it from `trig.jl`. This implies that Julia can call different methods of the same function to efficiently handle certain arguments based on the type inferenced by the compiler.

### @code_typed, @code_warntype, and @code_lowered

The function [code_typed](https://docs.julialang.org/en/v1/base/base/#Base.code_typed) explicitly infers the type of the argument used by the function, then returns what is called an array of type inferred lowered forms of the methods which match the generic function being passed and the inferred argument type. In the following example, `cos(x)` is the generic function and `0` and `0.0` are the arguments being passed by `cos(x)`. It is readily apparent that there are more processes involved when an (somewhat) explicit `float` is passed through the `cos(x)` function.

In [4]:
@code_typed cos(0)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, x)::Float64
└──      goto #3 if not false
2 ─      nothing::Nothing
3 ┄ %4 = invoke Base.Math.cos(%1::Float64)::Float64
└──      return %4
) => Float64

In [5]:
@code_typed cos(0.0)

CodeInfo(
1 ── %1   = Base.abs_float(x)::Float64
│    %2   = Base.lt_float(%1, 0.7853981633974483)::Bool
└───        goto #5 if not %2
2 ── %4   = Base.Math.sqrt_llvm(1.1102230246251565e-16)::Float64
│    %5   = Base.lt_float(%1, %4)::Bool
└───        goto #4 if not %5
3 ──        return 1.0
4 ── %8   = Base.mul_float(x, x)::Float64
│    %9   = Base.mul_float(%8, %8)::Float64
│    %10  = Base.muladd_float(%8, 2.480158728947673e-5, -0.001388888888887411)::Float64
│    %11  = Base.muladd_float(%8, %10, 0.0416666666666666)::Float64
│    %12  = Base.mul_float(%8, %11)::Float64
│    %13  = Base.muladd_float(%8, -1.1359647557788195e-11, 2.087572321298175e-9)::Float64
│    %14  = Base.muladd_float(%8, %13, -2.7557314351390663e-7)::Float64
│    %15  = Base.mul_float(%9, %9)::Float64
│    %16  = Base.mul_float(%15, %14)::Float64
│    %17  = Base.add_float(%12, %16)::Float64
│    %18  = Base.mul_float(0.5, %8)::Float64
│    %19  = Base.sub_float(1.0, %18)::Float64
│    %20  = Base.sub_float(1.0,

[code_warntype](https://docs.julialang.org/en/v1/stdlib/InteractiveUtils/#InteractiveUtils.code_warntype) prints out the generic function name, all the types of the variables involved (not just the argument), and the abstract syntax tree of the method process. Its primary use is to check whether or not there is potential type instability as the process runs. In the case of the examples below, the inferred types are stated clearly and there is no risk for type instability. (Although type instability may not necessarily be a bad thing for Julia as long as it correctly infers its argument types.)

In [6]:
@code_warntype cos(0)

Variables
  #self#::Core.Const(cos)
  x::Int64
  xf::Float64

Body::Float64
1 ─      (xf = Base.Math.float(x))
│   %2 = (x === xf)::Core.Const(false)
└──      goto #3 if not %2
2 ─      Core.Const(:(Core.tuple(x)))
│        Core.Const(:(Base.Math.MethodError(Base.Math.cos, %4)))
│        Core.Const(:(Base.Math.throw(%5)))
└──      Core.Const(:(goto %8))
3 ┄ %8 = Base.Math.cos(xf)::Float64
└──      return %8


In [7]:
@code_warntype cos(0.0)

Variables
  #self#::Core.Const(cos)
  x::Float64
  @_3::Int64
  y::Base.Math.DoubleFloat64
  n::Int64
  absx::Float64

Body::Float64
1 ──       Core.NewvarNode(:(@_3))
│          Core.NewvarNode(:(y))
│          Core.NewvarNode(:(n))
│          (absx = Base.Math.abs(x))
│    %5  = absx::Float64
│    %6  = ($(Expr(:static_parameter, 1)))(Base.Math.pi)::Core.Const(3.141592653589793)
│    %7  = (%6 / 4)::Core.Const(0.7853981633974483)
│    %8  = (%5 < %7)::Bool
└───       goto #5 if not %8
2 ── %10 = absx::Float64
│    %11 = Base.Math.eps($(Expr(:static_parameter, 1)))::Core.Const(2.220446049250313e-16)
│    %12 = ($(Expr(:static_parameter, 1)))(2.0)::Core.Const(2.0)
│    %13 = (%11 / %12)::Core.Const(1.1102230246251565e-16)
│    %14 = Base.Math.sqrt(%13)::Float64
│    %15 = (%10 < %14)::Bool
└───       goto #4 if not %15
3 ── %17 = ($(Expr(:static_parameter, 1)))(1.0)::Core.Const(1.0)
└───       return %17
4 ── %19 = Base.Math.cos_kernel(x)::Float64
└───       return %19
5 ── %21 = Base.

[code_lowered](https://docs.julialang.org/en/v1/base/base/#Base.code_lowered) is essentially the same as `code_warntype`, except it only shows the abstract syntax tree of the process.

In [7]:
@code_lowered cos(0)

CodeInfo(
1 ─      xf = Base.Math.float(x)
│   %2 = x === xf
└──      goto #3 if not %2
2 ─ %4 = Core.tuple(x)
│   %5 = Base.Math.MethodError(Base.Math.cos, %4)
│        Base.Math.throw(%5)
└──      goto #3
3 ┄ %8 = Base.Math.cos(xf)
└──      return %8
)

In [8]:
@code_lowered cos(0.0)

CodeInfo(
1 ──       Core.NewvarNode(:(@_3))
│          Core.NewvarNode(:(y))
│          Core.NewvarNode(:(n))
│          absx = Base.Math.abs(x)
│    %5  = absx
│    %6  = ($(Expr(:static_parameter, 1)))(Base.Math.pi)
│    %7  = %6 / 4
│    %8  = %5 < %7
└───       goto #5 if not %8
2 ── %10 = absx
│    %11 = Base.Math.eps($(Expr(:static_parameter, 1)))
│    %12 = ($(Expr(:static_parameter, 1)))(2.0)
│    %13 = %11 / %12
│    %14 = Base.Math.sqrt(%13)
│    %15 = %10 < %14
└───       goto #4 if not %15
3 ── %17 = ($(Expr(:static_parameter, 1)))(1.0)
└───       return %17
4 ── %19 = Base.Math.cos_kernel(x)
└───       return %19
5 ── %21 = Base.Math.isnan(x)
└───       goto #7 if not %21
6 ── %23 = ($(Expr(:static_parameter, 1)))(Base.Math.NaN)
└───       return %23
7 ── %25 = Base.Math.isinf(x)
└───       goto #9 if not %25
8 ── %27 = Base.Math.cos_domain_error(x)
└───       return %27
9 ── %29 = Base.Math.rem_pio2_kernel(x)
│    %30 = Base.indexed_iterate(%29, 1)
│          n = Core.ge

### @code_llvm

Lastly, [code_llvm](https://docs.julialang.org/en/v1/stdlib/InteractiveUtils/#InteractiveUtils.code_llvm) prints out the LLVM instructions to Julia's Just-In-Time compiler in order to executre the code. We can clearly see here the difference in libraries and modules used depending on the inferred variable types passed to the function.

In [9]:
@code_llvm cos(0)

;  @ math.jl:404 within `cos'
define double @julia_cos_1836(i64 signext %0) {
top:
;  @ math.jl:405 within `cos'
; ┌ @ float.jl:206 within `float'
; │┌ @ float.jl:191 within `AbstractFloat'
; ││┌ @ float.jl:94 within `Float64'
     %1 = sitofp i64 %0 to double
; └└└
;  @ math.jl:407 within `cos'
  %2 = call double @j_cos_1838(double %1)
  ret double %2
}


In [10]:
@code_llvm cos(0.0)

;  @ special/trig.jl:98 within `cos'
define double @julia_cos_1852(double %0) {
top:
  %1 = alloca { i64, [2 x double] }, align 8
;  @ special/trig.jl:99 within `cos'
; ┌ @ float.jl:441 within `abs'
   %2 = call double @llvm.fabs.f64(double %0)
; └
;  @ special/trig.jl:100 within `cos'
; ┌ @ float.jl:371 within `<'
   %3 = fcmp uge double %2, 0x3FE921FB54442D18
; └
  br i1 %3, label %L26, label %L4

L4:                                               ; preds = %top
;  @ special/trig.jl:101 within `cos'
; ┌ @ float.jl:371 within `<'
   %4 = fcmp uge double %2, 0x3E46A09E667F3BCD
; └
  br i1 %4, label %L8, label %L7

L7:                                               ; preds = %L26, %L4
  %merge = phi double [ 1.000000e+00, %L4 ], [ 0x7FF8000000000000, %L26 ]
;  @ special/trig.jl:102 within `cos'
  ret double %merge

L8:                                               ; preds = %L4
;  @ special/trig.jl:104 within `cos'
; ┌ @ special/trig.jl:144 within `cos_kernel'
; │┌ @ float.jl:332 within `

      %48 = icmp ult i32 %47, 17
; ││└└
    br i1 %48, label %L122, label %L88

L88:                                              ; preds = %L55
; │└
; │┌ @ special/rem_pio2.jl:78 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:332 within `*'
     %49 = fmul double %37, 0x3DD0B4611A600000
; │└└
; │┌ @ special/rem_pio2.jl:79 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %50 = fsub double %39, %49
; │└└
; │┌ @ special/rem_pio2.jl:80 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %51 = fsub double %39, %50
     %52 = fsub double %51, %49
; ││└
; ││┌ @ float.jl:339 within `muladd'
     %53 = fmul contract double %37, 0x3BA3198A2E037073
     %54 = fsub contract double %53, %52
; │└└
; │┌ @ special/rem_pio2.jl:81 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %55 = fsub double %50, %54
; │└└
; │┌ @ special/rem_pio2.jl:82 within `cody_waite_ext_pio2'
; ││┌ @ math.jl:1175 within `highword'
; │││┌ @ essentials.jl:422 within `reinterpret'
 

     %83 = fadd contract double %82, 0xBDE0B4611A626331
; │└└
; │┌ @ special/rem_pio2.jl:53 within `cody_waite_2c_pio2'
; ││┌ @ float.jl:329 within `-'
     %84 = fsub double %82, %83
; ││└
; ││┌ @ float.jl:339 within `muladd'
     %85 = fadd contract double %84, 0xBDE0B4611A626331
; │└└
   br label %L499.thread

L159:                                             ; preds = %L150
; └
; ┌ @ special/rem_pio2.jl:242 within `rem_pio2_kernel'
; │┌ @ special/rem_pio2.jl:51 within `cody_waite_2c_pio2'
; ││┌ @ float.jl:339 within `muladd'
     %86 = fadd contract double %0, 0x400921FB54400000
; │└└
; │┌ @ special/rem_pio2.jl:52 within `cody_waite_2c_pio2'
; ││┌ @ float.jl:339 within `muladd'
     %87 = fadd contract double %86, 0x3DE0B4611A626331
; │└└
; │┌ @ special/rem_pio2.jl:53 within `cody_waite_2c_pio2'
; ││┌ @ float.jl:329 within `-'
     %88 = fsub double %86, %87
; ││└
; ││┌ @ float.jl:339 within `muladd'
     %89 = fadd contract double %88, 0x3DE0B4611A626331
; │└└
   br label %L499.th

     %123 = fsub double %118, %122
     br label %L239

L239:                                             ; preds = %L232, %L205, %L172
     %value_phi3 = phi double [ %123, %L232 ], [ %97, %L172 ], [ %110, %L205 ]
     %value_phi4 = phi double [ %122, %L232 ], [ %96, %L172 ], [ %109, %L205 ]
     %value_phi5 = phi double [ %118, %L232 ], [ %95, %L172 ], [ %105, %L205 ]
; │└└
; │┌ @ special/rem_pio2.jl:92 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %124 = fsub double %value_phi5, %value_phi3
     %125 = fsub double %124, %value_phi4
; │└└
; │┌ @ special/rem_pio2.jl:93 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:239 within `unsafe_trunc'
     %126 = fptosi double %93 to i64
; │└└
   br label %L441

L249:                                             ; preds = %L170
; └
; ┌ @ special/rem_pio2.jl:255 within `rem_pio2_kernel'
; │┌ @ operators.jl:305 within `>'
; ││┌ @ float.jl:371 within `<'
     %127 = fcmp ule double %0, 0.000000e+00
; │└└
   br i1 %127, label %L258

     %159 = sub nsw i32 1025, %158
; │└└
; │┌ @ special/rem_pio2.jl:84 within `cody_waite_ext_pio2'
; ││┌ @ operators.jl:305 within `>'
; │││┌ @ int.jl:447 within `<' @ promotion.jl:361 @ int.jl:441
      %160 = icmp ult i32 %159, 50
; ││└└
    br i1 %160, label %L334, label %L327

L327:                                             ; preds = %L300
; │└
; │┌ @ special/rem_pio2.jl:86 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:332 within `*'
     %161 = fmul double %137, 0x3BA3198A2E000000
; │└└
; │┌ @ special/rem_pio2.jl:87 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %162 = fsub double %149, %161
; │└└
; │┌ @ special/rem_pio2.jl:88 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %163 = fsub double %149, %162
     %164 = fsub double %163, %161
; ││└
; ││┌ @ float.jl:339 within `muladd'
     %165 = fmul contract double %137, 0x397B839A252049C1
     %166 = fsub contract double %165, %164
; │└└
; │┌ @ special/rem_pio2.jl:89 within `cody_waite_ext_pio

; ││┌ @ float.jl:332 within `*'
     %194 = fmul double %182, 0x3DD0B4611A600000
; │└└
; │┌ @ special/rem_pio2.jl:79 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %195 = fsub double %184, %194
; │└└
; │┌ @ special/rem_pio2.jl:80 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %196 = fsub double %184, %195
     %197 = fsub double %196, %194
; ││└
; ││┌ @ float.jl:339 within `muladd'
     %198 = fmul contract double %182, 0x3BA3198A2E037073
     %199 = fsub contract double %198, %197
; │└└
; │┌ @ special/rem_pio2.jl:81 within `cody_waite_ext_pio2'
; ││┌ @ float.jl:329 within `-'
     %200 = fsub double %195, %199
; │└└
; │┌ @ special/rem_pio2.jl:82 within `cody_waite_ext_pio2'
; ││┌ @ math.jl:1175 within `highword'
; │││┌ @ essentials.jl:422 within `reinterpret'
      %201 = bitcast double %200 to i64
; │└└└
; │┌ @ special/rem_pio2.jl:83 within `cody_waite_ext_pio2'
; ││┌ @ int.jl:462 within `>>' @ int.jl:456
     %202 = lshr i64 %201, 52
     %203 =

  %.sroa.28.0176 = phi double [ %.sroa.28.0, %L441 ], [ %173, %L346 ], [ %177, %L353 ]
;  @ special/trig.jl:113 within `cos'
; ┌ @ special/trig.jl:136 within `cos_kernel'
; │┌ @ float.jl:332 within `*'
    %218 = fmul double %.sroa.28.0176, %.sroa.28.0176
; └└
; ┌ @ special/trig.jl:137 within `cos_kernel'
; │┌ @ float.jl:332 within `*'
    %219 = fmul double %218, %218
; └└
; ┌ @ special/trig.jl:138 within `cos_kernel'
; │┌ @ math.jl:131 within `evalpoly'
; ││┌ @ math.jl:132 within `macro expansion'
; │││┌ @ float.jl:339 within `muladd'
      %220 = fmul contract double %218, 0x3EFA01A019CB1590
      %221 = fadd contract double %220, 0xBF56C16C16C15177
      %222 = fmul contract double %218, %221
      %223 = fadd contract double %222, 0x3FA555555555554C
; │└└└
; │┌ @ float.jl:332 within `*'
    %224 = fmul double %218, %223
; │└
; │┌ @ math.jl:131 within `evalpoly'
; ││┌ @ math.jl:132 within `macro expansion'
; │││┌ @ float.jl:339 within `muladd'
      %225 = fmul contract double %218

      %295 = fadd contract double %294, 0xBF2A01A019C161D5
      %296 = fmul contract double %293, %295
      %297 = fadd contract double %296, 0x3F8111111110F8A6
      %298 = fmul contract double %293, 0x3DE5D93A5ACFD57C
      %299 = fadd contract double %298, 0xBE5AE5E68A2B9CEB
; │└└└
; │┌ @ operators.jl:560 within `*' @ float.jl:332
    %300 = fmul double %293, %292
    %301 = fmul double %300, %299
; │└
; │┌ @ float.jl:326 within `+'
    %302 = fadd double %301, %297
; └└
; ┌ @ special/trig.jl:73 within `sin_kernel'
; │┌ @ float.jl:332 within `*'
    %303 = fmul double %.sroa.28.0166201, %293
; └└
; ┌ @ special/trig.jl:74 within `sin_kernel'
; │┌ @ float.jl:332 within `*'
    %304 = fmul double %.sroa.40.0167200, 5.000000e-01
    %305 = fmul double %303, %302
; │└
; │┌ @ float.jl:329 within `-'
    %306 = fsub double %304, %305
; │└
; │┌ @ float.jl:332 within `*'
    %307 = fmul double %293, %306
; │└
; │┌ @ float.jl:329 within `-'
    %308 = fsub double %307, %.sroa.40.0167200
; │

## KR2: Julia constants

Here we will demonstrate the constant fixing of functions in Julia.
Let us define a function `g` such that

In [30]:
g(x) = x*cos(pi/2)

g (generic function with 1 method)

The function with name `g` is a generic function then containing a single method `x*cos(pi/2)`. Doing a `@code_warntype` and `@code_llvm` check on this function gives us

In [33]:
@code_llvm g(pi/2)

;  @ In[30]:1 within `g'
define double @julia_g_2145(double %0) {
top:
; ┌ @ float.jl:332 within `*'
   %1 = fmul double %0, 0x3C91A62633145C07
; └
  ret double %1
}


The LLVM code made by `@code_llvm` shows that `cos(pi/2)` is already parsed as the constant `0x3C91A62633145C07` even before the code is passed to the compiler. We can see the difference by defining another function `g_x` and passing it through `@code_llvm`.

In [22]:
gₓ(x) = x*cos(x)

gₓ (generic function with 1 method)

In [34]:
@code_llvm gₓ(pi/2)

;  @ In[22]:1 within `gₓ'
define double @"julia_g\E2\82\93_2147"(double %0) {
top:
  %1 = call double @j_cos_2149(double %0)
; ┌ @ float.jl:332 within `*'
   %2 = fmul double %1, %0
; └
  ret double %2
}


The LLVM output makes it clear that `cos(x)` is called and evaluated first before being multiplied to `x`. In this case, `cos(x = pi/2)` is not treated as a constant.

## KR3: Type inference and multiple dispatch

Julia makes a distinction between functions and methods. As shown in passing in KR1, the `cos(x)` function itself is the generic function attached to multiple methods. These methods can change based on the inferred type of the argument `x`. We can demonstrate this by defining our own function `forcecosine`.

In [54]:
forcecosine(x) = x*cos(x)
forcecosine(x::String) = x
forcecosine(x::Number) = x
forcecosine(x::Real) = x

forcecosine (generic function with 4 methods)

In [55]:
forcecosine(3.0)

3.0

## KR4: my_sum(x) vs sum(x), benchmarked with @time

## KR5: Plotting Mandelbrot in Julia

## KR6-7: Benchmarking Mandelbrot generation with @time macros